In [ ]:
!pip install transformers
!pip install datasets
!pip install torch
!pip install soundfile
!pip install librosa
from transformers import AutoFeatureExtractor, HubertForSequenceClassification
from datasets import load_dataset
import torch
import torch.nn.functional as F

In [ ]:
# from sklearn.metrics import precision_score, recall_score, f1_score
# from transformers import AutoFeatureExtractor
# from datasets import load_dataset
# from sklearn.ensemble import RandomForestClassifier
# from transformers import WhisperFeatureExtractor
# import numpy as np
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import GradientBoostingClassifier
# import torch
# import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
# Cargando el paquete de Colab para usar drive
from google.colab import drive

# Montar el colab en drive (Requiere autorización de Google Drive)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
ds_train = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/Tesis/acordes_completo/train")

Resolving data files:   0%|          | 0/30240 [00:00<?, ?it/s]

Computing checksums:  78%|#######7  | 23450/30240 [00:05<00:01, 4689.79it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
sampling_rate = 16000
feature_extractor = AutoFeatureExtractor.from_pretrained("ntu-spml/distilhubert")

max_duration = 4.0

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

In [ ]:
class ChordDataset(Dataset):
    def __init__(self, ds_train, feature_extractor, sampling_rate):
        self.ds_train = ds_train
        self.feature_extractor = feature_extractor
        self.sampling_rate = sampling_rate

    def __len__(self):
        return len(self.ds_train['train'])

    def __getitem__(self, idx):
        audio_array = self.ds_train['train'][idx]['audio']['array']
        inputs = self.feature_extractor(
            audio_array,
            sampling_rate=self.sampling_rate,
            return_tensors="pt"
        )
        return inputs['input_values'].squeeze(0)  # Ajusta según la salida de tu feature_extractor

# Suponiendo que `ds_train`, `feature_extractor` y `sampling_rate` están definidos
chord_dataset = ChordDataset(ds_train, feature_extractor, sampling_rate)

In [ ]:
batch_size = 32  # Ajusta el tamaño del lote según tu capacidad de memoria
audio_dataloader = DataLoader(chord_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
all_inputs = []

max_iterations = 1

for batch in enumerate(audio_dataloader):
    all_inputs.append(batch)

inputs = torch.cat(all_inputs, dim=0)

TypeError: expected Tensor as element 0 in argument 0, but got tuple

In [ ]:
# Crear una lista para almacenar los tensores procesados
all_inputs = []

# Iterar sobre todas las posiciones (asumiendo que ds_train['train'] tiene 30240 elementos)
for i in range(1):
    # Extraer las características de la muestra actual
    inputs = feature_extractor(
        ds_train['train'][i]['audio']['array'],
        sampling_rate=sampling_rate,
        return_tensors="pt"
    )
    # Agregar el tensor procesado a la lista
    all_inputs.append(inputs['input_values'])  # Ajusta esto según la estructura de salida de tu feature_extractor

inputs = torch.cat(all_inputs, dim=0)

In [ ]:
inputs

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0007, 0.0006, 0.0007]])

In [ ]:
# def preprocess_function(examples):
#     audio_arrays = [x["array"] for x in examples["audio"]]
#     inputs = feature_extractor(
#         audio_arrays,
#         sampling_rate=sampling_rate,
#         return_tensors="pt"
#     )
#     return inputs

In [ ]:
# ds_train_coded = preprocess_function(ds_train['train'])

In [ ]:
# from datasets import Dataset
# ds_train_coded_dataset = Dataset.from_dict(ds_train_coded)

In [ ]:
# ds_train_coded_dataset.save_to_disk('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspect')

Saving the dataset (0/16 shards):   0%|          | 0/30240 [00:00<?, ? examples/s]

In [ ]:
from datasets import load_from_disk
ds_train_prepoc = load_from_disk('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspect')

In [ ]:
ds_train_prepoc

Dataset({
    features: ['input_values'],
    num_rows: 30240
})

In [ ]:
print(type(ds_train_prepoc['input_values'][0]))

In [ ]:
from transformers import Wav2Vec2Processor, Wav2Vec2Model, Wav2Vec2FeatureExtractor
model = Wav2Vec2Model.from_pretrained("ntu-spml/distilhubert")

config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/94.0M [00:00<?, ?B/s]

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at ntu-spml/distilhubert and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'feature_projection.layer_norm.bias', 'feature_projection.layer_norm.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.nn.utils.rnn import pad_sequence
padded_inputs = pad_sequence(all_inputs, batch_first=True)
padded_inputs = padded_inputs.squeeze(1)

In [ ]:
padded_inputs.shape

torch.Size([1000, 64000])

In [ ]:
class AudioDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# Crear una instancia de tu Dataset
audio_dataset = AudioDataset(padded_inputs)

In [ ]:
# Crear un DataLoader
batch_size = 32  # Ajusta el tamaño del lote según tu capacidad de memoria
audio_dataloader = DataLoader(audio_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
all_features = []

with torch.no_grad():
    for batch in audio_dataloader:
        # Asegúrate de que el tensor tenga la forma correcta
        #batch = batch.unsqueeze(1)  # Si tu modelo espera la forma [batch_size, 1, sequence_length]
        features = model(batch).last_hidden_state
        all_features.append(features)

# Concatenar todas las características en un solo tensor si es necesario
all_features = torch.cat(all_features, dim=0)

In [ ]:
with torch.no_grad():
    features = model(padded_inputs).last_hidden_state

In [ ]:
all_features.shape

In [ ]:
pooled_features_max, _ = torch.max(all_features, dim=1)
pooled_features_max.shape

torch.Size([1000, 768])

In [ ]:
#ds_test = load_dataset("audiofolder", data_dir="/content/drive/MyDrive/Tesis/acordes_completo/test")

In [ ]:
#feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
# def prepare_dataset(audio_set):
#     audio = audio_set["audio"]
#     features = feature_extractor(
#         audio["array"], sampling_rate=audio["sampling_rate"], padding=True
#     )
#     return features

In [ ]:
# ds_train_prepoc = ds_train.map(prepare_dataset)
# ds_train_prepoc

In [ ]:
# #guardar los resultados del encode para no tener que preprocesar siempre
# ds_train_prepoc.save_to_disk('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspect')

In [ ]:
# ds_test_prepoc = ds_test.map(prepare_dataset)
# ds_test_prepoc

In [ ]:
#guardar los resultados del encode para no tener que preprocesar siempre
#ds_test_prepoc.save_to_disk('/content/drive/MyDrive/Tesis/dataOtherMoldels/testspect')

In [ ]:
#ds_train_prepoc = load_dataset('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspect')

In [ ]:
# import numpy as np
# import librosa
# import matplotlib.pyplot as plt

# example = ds_train_prepoc['train'][12]
# input_features = example["input_features"]

# plt.figure().set_figwidth(12)
# librosa.display.specshow(
#     np.asarray(input_features[0]),
#     x_axis="time",
#     y_axis="mel",
#     sr=feature_extractor.sampling_rate,
#     hop_length=feature_extractor.hop_length,
# )
# plt.colorbar()

In [ ]:
ds_train_prepoc = load_dataset('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspect')

Resolving data files:   0%|          | 0/17 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataX = []
labelY = []

In [ ]:
for i in range(0, 30243):
  dataX.append(ds_train_prepoc['train'][i]['input_features'][0][0])
  labelY.append(ds_train_prepoc['train'][i]['label'])
  print(i)

In [ ]:
dataX = np.array(dataX)
labelY = np.array(labelY)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataX, labelY, test_size=0.1, random_state=42)

Modelo 1. SVM

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#clfsvm = svm.SVC(kernel='linear', C=1.0)

# Entrena el modelo en los datos de entrenamiento
#clfsvm.fit(X_train, y_train)
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

for C in C_values:
    clfsvm = svm.SVC(kernel='linear', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')

    print(f'Modelo con Kernel lineal y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')


    clfsvm = svm.SVC(kernel='poly', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')


    print(f'Modelo con Kernel Polinómico y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')

    clfsvm = svm.SVC(kernel='rbf', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')


    print(f'Modelo con Kernel Radial y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')

# Realiza predicciones en los datos de prueba
#y_pred = clfsvm.predict(X_test)

# Evalúa el rendimiento del modelo (por ejemplo, usando la precisión)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel lineal y C = 0.001
Precision: 0.06420568478949866
Accuracy del modelo: 0.07
F1-score del modelo: 0.05
Recall: 0.07174739626384527


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Polinómico y C = 0.001
Precision: 0.003977336782345979
Accuracy del modelo: 0.02
F1-score del modelo: 0.01
Recall: 0.023309637956687057


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Modelo con Kernel Polinómico y C = 0.01
Precision: 0.12112819608489676
Accuracy del modelo: 0.05
F1-score del modelo: 0.04
Recall: 0.051082823607207804


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Radial y C = 0.01
Precision: 0.003545274350124635
Accuracy del modelo: 0.03
F1-score del modelo: 0.00
Recall: 0.026781286163002148
Modelo con Kernel lineal y C = 0.1
Precision: 0.37016937514339215
Accuracy del modelo: 0.37
F1-score del modelo: 0.37
Recall: 0.3749380062820301
Modelo con Kernel Polinómico y C = 0.1
Precision: 0.3486777612552677
Accuracy del modelo: 0.21
F1-score del modelo: 0.21
Recall: 0.20896015870391801


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Radial y C = 0.1
Precision: 0.1760820629064139
Accuracy del modelo: 0.12
F1-score del modelo: 0.09
Recall: 0.11572160687716979
Modelo con Kernel lineal y C = 1
Precision: 0.4729146464634029
Accuracy del modelo: 0.47
F1-score del modelo: 0.47
Recall: 0.4746239047776492
Modelo con Kernel Polinómico y C = 1
Precision: 0.5836107910874699
Accuracy del modelo: 0.57
F1-score del modelo: 0.57
Recall: 0.5695156224169284
Modelo con Kernel Radial y C = 1
Precision: 0.5303873078869632
Accuracy del modelo: 0.52
F1-score del modelo: 0.52
Recall: 0.5181021656472145
Modelo con Kernel lineal y C = 10
Precision: 0.5626574015067242
Accuracy del modelo: 0.56
F1-score del modelo: 0.56
Recall: 0.5612497933542734
Modelo con Kernel Polinómico y C = 10
Precision: 0.7850073582173209
Accuracy del modelo: 0.78
F1-score del modelo: 0.78
Recall: 0.782608695652174
Modelo con Kernel Radial y C = 10
Precision: 0.8074339416440509
Accuracy del modelo: 0.80
F1-score del modelo: 0.80
Recall: 0.8045958009

In [ ]:
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# precision = precision_score(y_test, y_pred, average='weighted')
# print(f'Precision: {precision}')
# print(f'Accuracy del modelo: {accuracy:.2f}')
# print(f'F1-score del modelo: {f1:.2f}')
# print(f'Recall: {recall}')

Modelo 2. Random Forest

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}


In [ ]:
# Realizamos la búsqueda en cuadrícula utilizando validación cruzada
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluamos el modelo con los mejores hiperparámetros en los datos de prueba
best_rf = grid_search.best_estimator_
accuracy = best_rf.score(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:424: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


Mejores hiperparámetros encontrados:
{'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Precisión en el conjunto de prueba: 0.78


In [ ]:
# rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_classifier.fit(X_train, y_train)

In [ ]:
# y_pred = rf_classifier.predict(X_test)

In [ ]:
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# precision = precision_score(y_test, y_pred, average='weighted')
# print(f'Precision: {precision}')
# print(f'Accuracy del modelo: {accuracy:.2f}')
# print(f'F1-score del modelo: {f1:.2f}')
# print(f'Recall: {recall}')

Modelo 3. Gradient Boosting

In [ ]:
##Solo para gradient boosting

#X_train, X_test, y_train, y_test = train_test_split(X_test, y_test, test_size=0.2, random_state=42)

In [ ]:
# clf = GradientBoostingClassifier(n_estimators=50, learning_rate=0.01, max_depth=10, random_state=42)
# #Entrena el modelo en los datos de entrenamiento
# clf.fit(X_train, y_train)

# #Realiza predicciones en los datos de prueba
# y_pred = clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy del modelo: {accuracy:.2f}')

In [ ]:
# clf = GradientBoostingClassifier(n_estimators=10, learning_rate=0.01, max_depth=None, random_state=42)
# #Entrena el modelo en los datos de entrenamiento
# clf.fit(X_train, y_train)

# #Realiza predicciones en los datos de prueba
# y_pred = clf.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy del modelo: {accuracy:.2f}')

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200]
}

xgb_model = xgb.XGBClassifier()
# Realizamos la búsqueda en cuadrícula utilizando validación cruzada
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluamos el modelo con los mejores hiperparámetros en los datos de prueba
best_gb = grid_search.best_estimator_
accuracy = best_gb.score(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

Fitting 5 folds for each of 27 candidates, totalling 135 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Mejores hiperparámetros encontrados:
{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 200}
Precisión en el conjunto de prueba: 0.77


In [ ]:
# accuracy = accuracy_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# precision = precision_score(y_test, y_pred, average='weighted')
# print(f'Precision: {precision}')
# print(f'Accuracy del modelo: {accuracy:.2f}')
# print(f'F1-score del modelo: {f1:.2f}')
# print(f'Recall: {recall}')

Datos de prueba

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
ds_test_prepoc = load_dataset('/content/drive/MyDrive/Tesis/dataOtherMoldels/testspect')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
datatestX = []
labeltestY = []

In [ ]:
for i in range(0, 12960):
  datatestX.append(ds_test_prepoc['train'][i]['input_features'][0][0])
  labeltestY.append(ds_test_prepoc['train'][i]['label'])
  print(i)

In [ ]:
datatestX = np.array(datatestX)
labeltestY = np.array(labeltestY)

SVM

In [ ]:
y_pred = clfsvm.predict(datatestX)

In [ ]:
accuracy = accuracy_score(labeltestY, y_pred)
f1 = f1_score(labeltestY, y_pred, average='weighted')
recall = recall_score(labeltestY, y_pred, average='weighted')
precision = precision_score(labeltestY, y_pred, average='weighted')
print(f'Precision: {precision}')
print(f'Accuracy del modelo: {accuracy:.2f}')
print(f'F1-score del modelo: {f1:.2f}')
print(f'Recall: {recall}')

Precision: 0.9318294277892939
Accuracy del modelo: 0.93
F1-score del modelo: 0.93
Recall: 0.93125


Random Forest

In [ ]:
 y_pred = best_rf.predict(datatestX)

In [ ]:
accuracy = accuracy_score(labeltestY, y_pred)
f1 = f1_score(labeltestY, y_pred, average='weighted')
recall = recall_score(labeltestY, y_pred, average='weighted')
precision = precision_score(labeltestY, y_pred, average='weighted')
print(f'Precision: {precision}')
print(f'Accuracy del modelo: {accuracy:.2f}')
print(f'F1-score del modelo: {f1:.2f}')
print(f'Recall: {recall}')

Precision: 0.9135379734071744
Accuracy del modelo: 0.91
F1-score del modelo: 0.91
Recall: 0.9121141975308642


Gradient Boosting

In [ ]:
y_pred = best_gb.predict(datatestX)

In [ ]:
accuracy = accuracy_score(labeltestY, y_pred)
f1 = f1_score(labeltestY, y_pred, average='weighted')
recall = recall_score(labeltestY, y_pred, average='weighted')
precision = precision_score(labeltestY, y_pred, average='weighted')
print(f'Precision: {precision}')
print(f'Accuracy del modelo: {accuracy:.2f}')
print(f'F1-score del modelo: {f1:.2f}')
print(f'Recall: {recall}')

Precision: 0.9434538195752787
Accuracy del modelo: 0.94
F1-score del modelo: 0.94
Recall: 0.9426697530864198


Empleando HuBERT

In [ ]:
feature_extractor = WhisperFeatureExtractor.from_pretrained("ntu-spml/distilhubert")

In [ ]:
def prepare_dataset(audio_set):
    audio = audio_set["audio"]
    features = feature_extractor(
        audio["array"], sampling_rate=audio["sampling_rate"], padding=True
    )
    return features

In [ ]:
ds_train_prepoc = ds_train.map(prepare_dataset)
ds_train_prepoc

Map:   0%|          | 0/30243 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['audio', 'label', 'input_features'],
        num_rows: 30243
    })
})

In [ ]:
ds_train_prepoc.save_to_disk('/content/drive/MyDrive/Tesis/dataOtherMoldels/trainspecthubert')

Saving the dataset (0/8 shards):   0%|          | 0/30243 [00:00<?, ? examples/s]

In [ ]:
train_encoded = load_dataset('/content/drive/MyDrive/Tesis/dataproces/train2')

Resolving data files:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
train_encoded

DatasetDict({
    train: Dataset({
        features: ['label', 'input_values', 'attention_mask'],
        num_rows: 30243
    })
})

In [ ]:
train_encoded['train'][30242]['label']

47

In [ ]:
train_encoded = train_encoded["train"].train_test_split(seed=42, shuffle=True, test_size=0.2)

In [ ]:
dataX = []
labelY = []

In [ ]:
for i in range(0, 4000):
  dataX.append(train_encoded['train'][i]['input_values'])
  labelY.append(train_encoded['train'][i]['label'])
  print(i)

In [ ]:
# Seleccionar solo las columnas 'question' y 'context' del conjunto de datos
selected_columns_dataset = train_encoded["train"].map(lambda example: {"input_values": example["input_values"]})
#print(selected_columns_dataset)

Map:   0%|          | 0/30243 [00:00<?, ? examples/s]

In [ ]:
selected_columns_dataset['input_values']

In [ ]:
dataX = np.array(dataX)
labelY = np.array(labelY)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataX, labelY, test_size=0.1, random_state=42)

In [ ]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#clfsvm = svm.SVC(kernel='linear', C=1.0)

# Entrena el modelo en los datos de entrenamiento
#clfsvm.fit(X_train, y_train)
C_values = [0.001, 0.01, 0.1, 1, 10, 100]

for C in C_values:
    clfsvm = svm.SVC(kernel='linear', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')

    print(f'Modelo con Kernel lineal y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')


    clfsvm = svm.SVC(kernel='poly', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')


    print(f'Modelo con Kernel Polinómico y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')

    clfsvm = svm.SVC(kernel='rbf', C=C)
    clfsvm.fit(X_train, y_train)
    y_pred = clfsvm.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    precision = precision_score(y_test, y_pred, average='weighted')


    print(f'Modelo con Kernel Radial y C = {C}')
    print(f'Precision: {precision}')
    print(f'Accuracy del modelo: {accuracy:.2f}')
    print(f'F1-score del modelo: {f1:.2f}')
    print(f'Recall: {recall}')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel lineal y C = 0.001
Precision: 0.04567343396177281
Accuracy del modelo: 0.06
F1-score del modelo: 0.04
Recall: 0.06214876033057851


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Polinómico y C = 0.001
Precision: 0.00015780342872754593
Accuracy del modelo: 0.01
F1-score del modelo: 0.00
Recall: 0.01256198347107438


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


...

Modelo con Kernel Radial y C = 0.01
Precision: 0.004724585041113981
Accuracy del modelo: 0.02
F1-score del modelo: 0.00
Recall: 0.0171900826446281
Modelo con Kernel lineal y C = 0.1
Precision: 0.29731714330806264
Accuracy del modelo: 0.29
F1-score del modelo: 0.27
Recall: 0.28793388429752065


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Polinómico y C = 0.1
Precision: 0.1190335822396778
Accuracy del modelo: 0.09
F1-score del modelo: 0.08
Recall: 0.09421487603305785


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Modelo con Kernel Radial y C = 0.1
Precision: 0.0949626740661724
Accuracy del modelo: 0.09
F1-score del modelo: 0.06
Recall: 0.08661157024793388
Modelo con Kernel lineal y C = 1
Precision: 0.41520391583323374
Accuracy del modelo: 0.41
F1-score del modelo: 0.40
Recall: 0.411900826446281
Modelo con Kernel Polinómico y C = 1
Precision: 0.2770884061836511
Accuracy del modelo: 0.26
F1-score del modelo: 0.25
Recall: 0.26380165289256197
Modelo con Kernel Radial y C = 1
Precision: 0.2655416092527853
Accuracy del modelo: 0.24
F1-score del modelo: 0.23
Recall: 0.24198347107438017
Modelo con Kernel lineal y C = 10
Precision: 0.5090938433364379
Accuracy del modelo: 0.51
F1-score del modelo: 0.50
Recall: 0.5051239669421488
Modelo con Kernel Polinómico y C = 10
Precision: 0.5774807278737988
Accuracy del modelo: 0.55
F1-score del modelo: 0.55
Recall: 0.5530578512396694
Modelo con Kernel Radial y C = 10
Precision: 0.544637332277095
Accuracy del modelo: 0.53
F1-score del modelo: 0.52
Recall: 0.52727272

In [ ]:
rf = RandomForestClassifier()

param_grid = {
    'n_estimators': [10, 50, 100],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}


In [ ]:
# Realizamos la búsqueda en cuadrícula utilizando validación cruzada
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')
# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluamos el modelo con los mejores hiperparámetros en los datos de prueba
best_rf = grid_search.best_estimator_
accuracy = best_rf.score(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: ignored

In [ ]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train, label=y_train)

param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.1, 0.01, 0.001],
    'n_estimators': [50, 100, 200]
}

xgb_model = xgb.XGBClassifier()
# Realizamos la búsqueda en cuadrícula utilizando validación cruzada
grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='accuracy')

# Ajustamos el modelo a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros encontrados:")
print(grid_search.best_params_)

# Evaluamos el modelo con los mejores hiperparámetros en los datos de prueba
best_gb = grid_search.best_estimator_
accuracy = best_gb.score(X_test, y_test)
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}")